<a href="https://colab.research.google.com/github/Saria003/Saria003/blob/main/kivy_besm_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-fa-0.42.zip -O vosk-model-fa-0.42.zip

!unzip vosk-model-fa-0.42.zip

!mv vosk-model-fa-0.42 models

!rm vosk-model-fa-0.42.zip


# FAILED
# !mkdir -p libs
# !pip download vosk==0.3.45 -d libs/

In [ ]:
# 0
from google.colab import drive
drive.mount('/content/drive')

import os, textwrap

DRIVE_ROOT = '/content/drive/MyDrive/voiceapp'
PROJECT_ROOT = f'{DRIVE_ROOT}/project'
CACHE_ROOT   = f'{DRIVE_ROOT}/cache'
OUTPUTS      = f'{DRIVE_ROOT}/outputs'

os.makedirs(PROJECT_ROOT, exist_ok=True)
os.makedirs(CACHE_ROOT, exist_ok=True)
os.makedirs(OUTPUTS + '/apk', exist_ok=True)

# SDK/NDK/Gradle caches on Drive (persist across sessions)
os.makedirs(f'{CACHE_ROOT}/android-sdk', exist_ok=True)
os.makedirs(f'{CACHE_ROOT}/android-ndk-r25b', exist_ok=True)
os.makedirs(f'{CACHE_ROOT}/.gradle', exist_ok=True)

print('PROJECT_ROOT:', PROJECT_ROOT)
print('CACHE_ROOT  :', CACHE_ROOT)
print('OUTPUTS     :', OUTPUTS)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_ROOT: /content/drive/MyDrive/voiceapp/project
CACHE_ROOT  : /content/drive/MyDrive/voiceapp/cache
OUTPUTS     : /content/drive/MyDrive/voiceapp/outputs


In [ ]:
# # 1
# !sudo apt-get update -y
# !sudo apt-get install -y \
#   python3.10 python3.10-venv python3.10-dev \
#   openjdk-17-jdk git unzip zlib1g-dev libncurses5-dev \
#   libffi-dev libssl-dev libsqlite3-dev libbz2-dev liblzma-dev \
#   build-essential autoconf automake libtool pkg-config scons

# # Create venv for Buildozer
# !VENV=/content/venv310
# !python3.10 -m venv "$VENV"
# !"$VENV/bin/python" -m pip install --upgrade pip wheel setuptools "cython==0.29.36" buildozer
# "$VENV/bin/python" -V
# "$VENV/bin/buildozer" --version

import os

# Define the virtual environment path
VENV = '/content/venv310'

# Install system dependencies
!sudo apt-get update -y
!sudo apt-get install -y \
  python3.10 python3.10-venv python3.10-dev \
  openjdk-17-jdk git unzip zlib1g-dev libncurses5-dev \
  libffi-dev libssl-dev libsqlite3-dev libbz2-dev liblzma-dev \
  build-essential autoconf automake libtool pkg-config scons

# Create virtual environment for Buildozer
!python3.10 -m venv {VENV}
!{VENV}/bin/python -m pip install --upgrade pip wheel setuptools "cython==0.29.36" buildozer

# Verify if Buildozer was installed correctly
!{VENV}/bin/python -V
!{VENV}/bin/buildozer --version

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,823 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 https:/

In [ ]:
# # 2
# # Persist Gradle cache and Android SDK/NDK on Drive
# !export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
# !export ANDROIDSDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"
# !export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
# !export ANDROID_HOME="$ANDROIDSDK"
# !export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"

# !mkdir -p "$GRADLE_USER_HOME" "$ANDROIDSDK" "$ANDROIDNDK"

# # Make sure they stick for this session
# !echo "export GRADLE_USER_HOME=$GRADLE_USER_HOME" >> ~/.bashrc
# !echo "export ANDROIDSDK=$ANDROIDSDK" >> ~/.bashrc
# !echo "export ANDROIDNDK=$ANDROIDNDK" >> ~/.bashrc
# !echo "export ANDROID_HOME=$ANDROID_HOME" >> ~/.bashrc
# !echo "export JAVA_HOME=$JAVA_HOME" >> ~/.bashrc


import os

os.environ['GRADLE_USER_HOME'] = '/content/drive/MyDrive/voiceapp/cache/.gradle'
os.environ['ANDROIDSDK']       = '/content/drive/MyDrive/voiceapp/cache/android-sdk'
os.environ['ANDROIDNDK']       = '/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b'
os.environ['ANDROID_HOME']     = os.environ['ANDROIDSDK']
os.environ['JAVA_HOME']        = '/usr/lib/jvm/java-17-openjdk-amd64'

# make sure directories exist
for d in (os.environ['GRADLE_USER_HOME'], os.environ['ANDROIDSDK'], os.environ['ANDROIDNDK']):
    os.makedirs(d, exist_ok=True)

# quick check they’re visible to future shell commands
!echo "GRADLE_USER_HOME=$GRADLE_USER_HOME"
!echo "ANDROIDSDK=$ANDROIDSDK"
!echo "ANDROIDNDK=$ANDROIDNDK"
!echo "JAVA_HOME=$JAVA_HOME"


GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64


In [ ]:
# 3: upload other files(all except main.py, buildozer.spec, audio_engine.py) to drive
from google.colab import files, output
upload = files.upload()  # pick: main.kv, utils_persian.py, camera_popup.py, pyaudio_stub.py, etc.

import os, io
for name, data in upload.items():
    with open(f"{PROJECT_ROOT}/{name}", "wb") as f:
        f.write(data)
print("Saved:", list(upload.keys()))

Saving api_helpers.py to api_helpers.py
Saving camera_popup.py to camera_popup.py
Saving main.kv to main.kv
Saving pyaudio_stub.py to pyaudio_stub.py
Saving used_bars.json to used_bars.json
Saving utils_persian.py to utils_persian.py
Saved: ['api_helpers.py', 'camera_popup.py', 'main.kv', 'pyaudio_stub.py', 'used_bars.json', 'utils_persian.py']


In [ ]:
!ls

drive  sample_data  venv310


In [ ]:
# 4
import os, pathlib, textwrap, json
ROOT = "/content/drive/MyDrive/voiceapp/project"
os.makedirs(ROOT, exist_ok=True)

# ---------- main.py (updated to place future import first; no fragile sys.path hacks)
main_py = r'''
from __future__ import annotations
import os, json, threading, logging, ctypes
import jdatetime

from kivy.app import App
from kivy.lang import Builder
from kivy.clock import Clock
from kivy.uix.screenmanager import ScreenManager, Screen, NoTransition
from kivy.properties import StringProperty, NumericProperty, BooleanProperty
from kivy.uix.popup import Popup
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button
from kivy.core.window import Window

from utils_persian import PersianLabel, fa
import api_helpers  # keep this if you have it; else comment out
from audio_engine import AudioEngine
from camera_popup import CameraCapturePopup

Window.clearcolor = (1, 1, 1, 1)

def make_white_popup(lines_raw: list[str], width=420, height=280, with_ok=True) -> Popup:
    root = BoxLayout(orientation='vertical', padding=(16, 16), spacing=14)
    for t in lines_raw:
        lbl = PersianLabel(source_text=t)
        lbl.halign = 'center'
        lbl.text_size = (width - 32, None)
        root.add_widget(lbl)
    if with_ok:
        btn = Button(text=fa("باشه"), size_hint_y=None, height=48)
        root.add_widget(btn)
    pop = Popup(title='', content=root, size_hint=(None, None), size=(width, height), auto_dismiss=False)
    if with_ok:
        btn.bind(on_release=lambda *_: pop.dismiss())
    return pop

def make_processing_popup(msg="در حال پردازش...", width=360, height=160) -> Popup:
    root = BoxLayout(orientation='vertical', padding=(16, 16), spacing=12)
    lbl = PersianLabel(source_text=msg)
    lbl.halign = 'center'
    lbl.text_size = (width - 32, None)
    lbl.color = (1, 1, 1, 1)
    root.add_widget(lbl)
    pop = Popup(title='', content=root, size_hint=(None, None), size=(width, height), auto_dismiss=False)
    return pop

class LoginScreen(Screen):
    busy = BooleanProperty(False)
    status_text = StringProperty(" ")

    def do_login(self):
        if self.busy:
            return
        user = self.ids.user.text.strip()
        pwd = self.ids.pwd.text.strip()
        if not user or not pwd:
            self.status_text = "نام کاربری و گذرواژه لازم است"
            return

        self.busy = True
        pop = make_processing_popup("در حال ورود ...", width=360, height=160)
        pop.open()

        def worker():
            ok = True
            try:
                ok = api_helpers.api_login(user, pwd, retries=3, delay=2, timeout=9)
            except Exception:
                logging.exception("api_login failed")
            def done(_=None):
                self.busy = False
                try: pop.dismiss()
                except Exception: pass
                if ok:
                    self.status_text = "ورود موفق بود"
                    self.manager.current = "barinfo"
                else:
                    self.status_text = "نام کاربری یا گذرواژه نادرست است یا سرور پاسخ نمی‌دهد"
            Clock.schedule_once(done, 0)
        threading.Thread(target=worker, daemon=True).start()

class BarInfoScreen(Screen):
    info_text = StringProperty(" ")

    def confirm_info(self):
        num = self.ids.number.text.strip()
        bar = self.ids.bar.text.strip()
        if not num or not bar:
            self.info_text = "لطفاً همه مقادیر را وارد کنید."
            return
        try:
            number = int(num)
            if number < 0:
                raise ValueError
        except Exception:
            self.info_text = "تعداد قطعه باید عدد باشد."
            return

        try:
            with open("used_bars.json", "r", encoding="utf-8") as f:
                used = json.load(f)
        except Exception:
            used = {}

        shamsi_short = jdatetime.datetime.now().strftime("%y%m%d")
        barnum = f"{bar}{shamsi_short}"
        used.setdefault(shamsi_short, [])
        if bar in used[shamsi_short]:
            dups = ", ".join(used[shamsi_short])
            self.info_text = f"شماره {bar} قبلاً برای امروز ثبت شده!\nشماره‌های واردشده امروز: {dups}"
            return

        used[shamsi_short].append(bar)
        try:
            with open("used_bars.json", "w", encoding="utf-8") as f:
                json.dump(used, f, ensure_ascii=False, indent=2)
        except Exception:
            pass

        app = App.get_running_app()
        app.number = number
        app.barnum = barnum
        self.info_text = f"شماره بار نهایی: {barnum}"
        Clock.schedule_once(lambda *_: setattr(self.manager, 'current', 'recorder'), 0.25)

class RecorderScreen(Screen):
    timer_text = StringProperty("درحال ضبط: 00:00")
    count_text = StringProperty("تعداد واژه بسم الله: 0")
    is_recording = BooleanProperty(False)

    def on_pre_enter(self):
        app = App.get_running_app()
        if app._timer_ev is None:
            app._timer_ev = Clock.schedule_interval(app._update_timer, 1.0)
        if not app._two_hour_ev:
            app._two_hour_ev = Clock.schedule_interval(lambda dt: app.show_camera_popup(), 7200.0)
        self._reset_summary()
        self._set_buttons(start=True, stop=False, process=False, finish=False)

    def _set_buttons(self, *, start, stop, process, finish):
        self.ids.start_btn.disabled   = not start
        self.ids.stop_btn.disabled    = not stop
        self.ids.process_btn.disabled = not process
        self.ids.finish_btn.disabled  = not finish

    def _format_time_line(self, recording: bool) -> str:
        app = App.get_running_app()
        secs = app.engine.current_elapsed_seconds(now=True)
        m, s = divmod(secs, 60)
        return f"{'درحال ضبط' if recording else 'ضبط متوقف شد'}: {m:02d}:{s:02d}"

    def start_recording(self):
        app = App.get_running_app()
        app.engine.on_count_update = lambda total: Clock.schedule_once(lambda *_: self.set_count(total), 0)
        try:
            ok = app.engine.start_or_resume()
        except Exception:
            logging.exception("start_or_resume failed")
            ok = False
        if ok:
            self.is_recording = True
            self._set_buttons(start=False, stop=True, process=False, finish=False)
            self.timer_text = self._format_time_line(recording=True)

    def stop_recording(self):
        app = App.get_running_app()
        try:
            app.engine.pause()
        except Exception:
            logging.exception("pause() failed (ignored)")
        self.is_recording = False
        self._set_buttons(start=True, stop=False, process=True, finish=True)
        self.timer_text = self._format_time_line(recording=False)

    def process_final(self):
        app = App.get_running_app()
        try:
            app.engine.pause()
        except Exception:
            pass
        self.is_recording = False

        def worker():
            try:
                results = app.engine.finalize_and_get_results(wait=True)
                results["number"] = app.number or 0
                results["barnum"] = app.barnum or ""
                try:
                    api_helpers.send_results_to_api(results)
                except Exception:
                    logging.exception("send_results_to_api failed")
                total = results.get("total_count", 0)
                st = results.get("start_time_shamsi", "")
                et = results.get("end_time_shamsi", "")
                def update_ui(_=None):
                    self.set_count(total)
                    self._show_summary(start_txt=st, end_txt=et, avg_txt="")
                    self._set_buttons(start=True, stop=False, process=True, finish=True)
                    self.timer_text = self._format_time_line(recording=False)
                Clock.schedule_once(update_ui, 0)
            except Exception:
                logging.exception("finalize_and_get_results failed")
        threading.Thread(target=worker, daemon=True).start()

    def calculate_average(self):
        app = App.get_running_app()
        total = app.engine.total_count
        dur = app.engine.current_elapsed_seconds(now=True)
        avg = (dur / total) if total > 0 and dur > 0 else 0.0
        self.ids.avg_lbl.source_text = f"میانگین زمان بین واژه‌ها: {avg:.2f} ثانیه"

    def finish(self):
        app = App.get_running_app()
        try:
            app.engine.pause()
        except Exception:
            pass
        app.engine = AudioEngine(on_count_update=None)
        self._reset_summary()
        self.set_count(0)
        self.timer_text = "درحال ضبط: 00:00"
        self._set_buttons(start=True, stop=False, process=False, finish=False)
        self.manager.current = "barinfo"

    def _reset_summary(self):
        self.ids.summary_card.opacity = 0
        self.ids.summary_card.disabled = True
        self.ids.start_lbl.source_text = ""
        self.ids.end_lbl.source_text = ""
        self.ids.avg_lbl.source_text = ""

    def _show_summary(self, start_txt: str, end_txt: str, avg_txt: str):
        self.ids.start_lbl.source_text = f"زمان شروع: {start_txt}" if start_txt else ""
        self.ids.end_lbl.source_text = f"زمان پایان: {end_txt}" if end_txt else ""
        self.ids.avg_lbl.source_text = avg_txt or ""
        self.ids.summary_card.opacity = 1
        self.ids.summary_card.disabled = False

    def set_count(self, total: int):
        self.count_text = f"تعداد واژه بسم الله: {total}"

class BesmApp(App):
    number = NumericProperty(0)
    barnum = StringProperty("")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")
        self.engine = AudioEngine(on_count_update=None)
        self._timer_ev = None
        self._two_hour_ev = None

    def build(self):
        Builder.load_file("main.kv")
        sm = ScreenManager(transition=NoTransition())
        sm.add_widget(LoginScreen(name="login"))
        sm.add_widget(BarInfoScreen(name="barinfo"))
        sm.add_widget(RecorderScreen(name="recorder"))
        sm.current = "login"
        try:
            user32 = ctypes.windll.user32  # ignored on Android
            sw, sh = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)
            ww, wh = 420, 640
            Window.size = (ww, wh); Window.left = (sw - ww) // 2; Window.top = (sh - wh) // 2
        except Exception:
            Window.size = (420, 640)
        return sm

    def _update_timer(self, dt):
        scr = self.root.get_screen("recorder")
        if scr and scr.is_recording:
            scr.timer_text = scr._format_time_line(recording=True)

    def show_camera_popup(self):
        if self.root.current != "recorder":
            return
        CameraCapturePopup().open()

if __name__ == "__main__":
    BesmApp().run()
'''.strip()

# ---------- audio_engine.py (APM + fallback DSP; Android NS/AGC/AEC + IIRs if no APM)
audio_engine_py = r"""
from __future__ import annotations
import os, re, time, threading, json, logging
from array import array
from queue import Queue, Empty
from typing import Callable, Optional

import jdatetime
from vosk import Model, KaldiRecognizer

CHANNELS = 1
RATE = 16000
CHUNK = 1024  # capture chunk; processing uses 10 ms = 160 samples

VOSK_MODEL_PATH = os.path.join(os.path.dirname(__file__), "models")

MIN_CONF   = float(os.environ.get("MIN_CONF", "0.60"))
DEBOUNCE_S = float(os.environ.get("DEBOUNCE_S", "0.35"))
MERGE_GAP_S= float(os.environ.get("MERGE_GAP_S", "0.50"))

GRAMMAR = [
    "[unk]",
    "بسم الله", "بسم‌الله",
    "بسم", "الله",
    "بسمیلو", "بسملا", "بسمی", "بیسمه", "بیسمیلا", "بسمل", "بس",
]

def _normalize_fa(s: str) -> str:
    s = (s or "")
    s = s.replace("\u200c", " ")
    s = s.replace("ي", "ی").replace("ك", "ک")
    s = re.sub(r"\s+", " ", s).strip()
    return s

TARGETS = {_normalize_fa(t) for t in GRAMMAR}

# --- Android classes via jnius (lazy so it doesn't crash on host) ---
def _android_classes():
    from jnius import autoclass
    AudioRecord = autoclass('android.media.AudioRecord')
    AudioFormat = autoclass('android.media.AudioFormat')
    MediaRecorder = autoclass('android.media.MediaRecorder')
    NoiseSuppressor = autoclass('android.media.audiofx.NoiseSuppressor')
    AutomaticGainControl = autoclass('android.media.audiofx.AutomaticGainControl')
    AcousticEchoCanceler = autoclass('android.media.audiofx.AcousticEchoCanceler')
    return (AudioRecord, AudioFormat, MediaRecorder,
            NoiseSuppressor, AutomaticGainControl, AcousticEchoCanceler)

AudioRecord = AudioFormat = MediaRecorder = None
NoiseSuppressor = AutomaticGainControl = AcousticEchoCanceler = None

# --- Optional WebRTC APM (Java) hook; safe if missing ---
APM = None
def _apm_java():
    global APM
    if APM is not None:
        return APM
    try:
        from jnius import autoclass
        APM = autoclass('org.saria.apm.ApmBridge')  # will be False if class not found
    except Exception:
        APM = False
    return APM

# --- RNNoise via ctypes (Android linker finds librnnoise.so in libs/arm64-v8a) ---
class _RnWrapper:
    def __init__(self):
        import ctypes as C
        self.C = C
        self.lib = None
        self.st = None
        self.ok = False
        self._load()

    def _load(self):
        C = self.C
        # Try default soname first (works on Android)
        candidates = [
            "librnnoise.so",
            os.path.join(os.path.dirname(__file__), "libs", "arm64-v8a", "librnnoise.so"),
        ]
        for path in candidates:
            try:
                lib = C.CDLL(path)
                # Declare signatures carefully
                lib.rnnoise_create.argtypes  = [C.c_void_p]          # const RNNModel* (NULL -> built-in)
                lib.rnnoise_create.restype   = C.c_void_p
                lib.rnnoise_destroy.argtypes = [C.c_void_p]
                lib.rnnoise_process_frame.argtypes = [
                    C.c_void_p,
                    C.POINTER(C.c_float),
                    C.POINTER(C.c_float),
                ]
                lib.rnnoise_process_frame.restype = C.c_float  # (returns VAD prob)
                self.lib = lib
                logging.info("RNNoise loaded from %s", path)
                break
            except Exception:
                pass
        self.ok = self.lib is not None

    def create(self):
        if not self.ok or self.st:
            return bool(self.st)
        try:
            self.st = self.lib.rnnoise_create(self.C.c_void_p(0))  # pass NULL
        except Exception:
            self.st = None
            logging.exception("RNNoise create failed")
        return bool(self.st)

    def destroy(self):
        if self.ok and self.st:
            try: self.lib.rnnoise_destroy(self.st)
            except Exception: pass
        self.st = None

    def process_10ms_16k(self, x16: array) -> array:
        # x16: array('h') of length 160. Returns array('h') length 160.
        C = self.C
        if not (self.ok and self.st and len(x16) == 160):
            return x16

        # Upsample 16k -> 48k (×3) with simple linear interpolation (good enough for denoiser)
        in48  = (C.c_float * 480)()
        for i in range(160):
            a = float(x16[i]) / 32768.0
            b = float(x16[i+1]) / 32768.0 if i+1 < 160 else a
            in48[3*i+0] = a
            in48[3*i+1] = (2*a + b) / 3.0
            in48[3*i+2] = (a + 2*b) / 3.0

        out48 = (C.c_float * 480)()
        try:
            self.lib.rnnoise_process_frame(self.st, out48, in48)
        except Exception:
            return x16

        # Downsample 48k -> 16k by averaging each triplet (less aliasing than pick-one)
        y16 = array('h', [0]*160)
        for i in range(160):
            m = (out48[3*i] + out48[3*i+1] + out48[3*i+2]) / 3.0  # -1..1
            m = 1.0 if m > 1.0 else (-1.0 if m < -1.0 else m)
            y16[i] = int(m * 32767.0)
        return y16

try:
    RN = _RnWrapper()
except Exception:
    RN = None

# --- tiny IIR fallback filters ---
class OnePoleHPF:
    def __init__(self, fc: float, fs: float):
        rc = 1.0/(2.0*3.141592653589793*fc); dt = 1.0/fs
        self.a = rc/(rc+dt); self.prev_x = 0.0; self.prev_y = 0.0
    def process_inplace(self, xh: array):
        a = self.a; px = self.prev_x; py = self.prev_y
        for i in range(len(xh)):
            x = float(xh[i]); y = a*(py + x - px)
            xh[i] = int(32767 if y>32767 else (-32768 if y<-32768 else y))
            px, py = x, y
        self.prev_x, self.prev_y = px, py

class OnePoleLPF:
    def __init__(self, fc: float, fs: float):
        rc = 1.0/(2.0*3.141592653589793*fc); dt = 1.0/fs
        self.alpha = dt/(rc+dt); self.prev_y = 0.0
    def process_inplace(self, xh: array):
        a = self.alpha; y = self.prev_y
        for i in range(len(xh)):
            xi = float(xh[i]); y = y + a*(xi - y)
            xh[i] = int(32767 if y>32767 else (-32768 if y<-32768 else y))
        self.prev_y = y

class PreEmphasis:
    def __init__(self, coeff: float = 0.97):
        self.a = coeff; self.prev = 0.0
    def process_inplace(self, xh: array):
        a = self.a; p = self.prev
        for i in range(len(xh)):
            x = float(xh[i]); y = x - a*p; p = x
            xh[i] = int(32767 if y>32767 else (-32768 if y<-32768 else y))
        self.prev = p

class AudioEngine:
    def __init__(self, on_count_update: Optional[Callable[[int], None]] = None):
        self.on_count_update = on_count_update
        self._stream = None
        self._buffer_size = 0
        self._record_evt = threading.Event()
        self._recording_thread = None
        self._processing_thread = None
        self.audio_queue: Queue[list[bytes]] = Queue()
        self.total_count = 0

        self.start_time_epoch = 0.0
        self.end_time_epoch = 0.0
        self.start_time_shamsi = ""
        self.end_time_shamsi = ""

        self._has_started = False
        self._paused = False
        self._pause_started = 0.0
        self._pause_accum = 0.0

        self._model: Optional[Model] = None
        self._recognizer: Optional[KaldiRecognizer] = None
        self._finalized = False
        self._last_hit_time: Optional[float] = None

        self.number: Optional[int] = None
        self.barnum: Optional[str] = None

        self._pre = PreEmphasis(0.97)
        self._hpf = OnePoleHPF(120.0, RATE)
        self._lpf = OnePoleLPF(3800.0, RATE)

        self._fx = {}
        self._apm = None          # Java APM or False
        self._rn_active = False
        self._residual = array('h')  # carry remainder <160 samples between loops

    def load_model(self):
        if self._model is None:
            if not os.path.isdir(VOSK_MODEL_PATH):
                raise FileNotFoundError(f"Vosk model not found at: {VOSK_MODEL_PATH}")
            logging.info("Loading Vosk model from %s ...", VOSK_MODEL_PATH)
            self._model = Model(VOSK_MODEL_PATH)

    def _ensure_recognizer(self):
        if self._recognizer is None:
            try:
                self._recognizer = KaldiRecognizer(self._model, RATE, json.dumps(GRAMMAR))
                self._recognizer.SetWords(True)
            except Exception as e:
                logging.error("Recognizer with grammar failed (%s). Falling back to free vocab.", e)
                self._recognizer = KaldiRecognizer(self._model, RATE)
                self._recognizer.SetWords(True)
            self._finalized = False
            self._last_hit_time = None

    def _ensure_apm(self):
        if self._apm is not None:
            return
        APMClass = _apm_java()
        if not APMClass:
            self._apm = False
            return
        try:
            self._apm = APMClass(True, 2, True, True, False, RATE)  # ns HIGH, agc, hpf, aec=False
            logging.info("WebRTC APM initialized")
        except Exception:
            logging.exception("APM init failed; ignoring")
            self._apm = False

    def _open_stream(self):
        global AudioRecord, AudioFormat, MediaRecorder
        global NoiseSuppressor, AutomaticGainControl, AcousticEchoCanceler
        if AudioRecord is None:
            (AudioRecord, AudioFormat, MediaRecorder,
             NoiseSuppressor, AutomaticGainControl, AcousticEchoCanceler) = _android_classes()

        min_buf = AudioRecord.getMinBufferSize(
            RATE, AudioFormat.CHANNEL_IN_MONO, AudioFormat.ENCODING_PCM_16BIT
        )
        buf_sz = max(int(min_buf), 4 * CHUNK)

        self._stream = AudioRecord(
            MediaRecorder.AudioSource.VOICE_RECOGNITION,
            RATE, AudioFormat.CHANNEL_IN_MONO, AudioFormat.ENCODING_PCM_16BIT, buf_sz
        )
        self._stream.startRecording()
        self._buffer_size = buf_sz

        # RN state if available
        if isinstance(RN, _RnWrapper):
            self._rn_active = RN.create()
        else:
            self._rn_active = False

        # Android FX: AGC+AEC always, NS only if RN not active (avoid double-denoise)
        try:
            session = self._stream.getAudioSessionId()
            self._fx = {}
            if AutomaticGainControl.isAvailable():
                self._fx['agc'] = AutomaticGainControl.create(session);
                if self._fx['agc']: self._fx['agc'].setEnabled(True)
            if AcousticEchoCanceler.isAvailable():
                self._fx['aec'] = AcousticEchoCanceler.create(session);
                if self._fx['aec']: self._fx['aec'].setEnabled(True)
            if (not self._rn_active) and NoiseSuppressor.isAvailable():
                self._fx['ns'] = NoiseSuppressor.create(session);
                if self._fx['ns']: self._fx['ns'].setEnabled(True)
            logging.info("Audio FX enabled: %s (RN=%s)", list(self._fx.keys()), self._rn_active)
        except Exception:
            logging.exception("Audio FX setup failed (ignored)")

    def start_or_resume(self):
        if self._record_evt.is_set():
            return False
        self.load_model()
        self._ensure_recognizer()

        if not self._has_started:
            self.total_count = 0
            self.start_time_epoch = time.time()
            self.start_time_shamsi = jdatetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
            self.end_time_epoch = 0.0
            self.end_time_shamsi = ""
            self._pause_accum = 0.0
            self._pause_started = 0.0
            self._paused = False
            self._has_started = True
        else:
            self._paused = False
            if self._pause_started:
                self._pause_accum += max(0.0, time.time() - self._pause_started)
                self._pause_started = 0.0

        self._open_stream()
        self._record_evt.set()
        self._recording_thread = threading.Thread(target=self._record_loop, daemon=True)
        self._recording_thread.start()
        if not self._processing_thread or not self._processing_thread.is_alive():
            self._processing_thread = threading.Thread(target=self._process_loop, daemon=True)
            self._processing_thread.start()
        logging.info("🎙️ start/resume")
        return True

    def pause(self):
        if not self._record_evt.is_set():
            return False
        self._record_evt.clear()
        self._paused = True
        self._pause_started = time.time()
        self.end_time_shamsi = jdatetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
        try:
            if self._recording_thread and self._recording_thread.is_alive():
                self._recording_thread.join(timeout=1.0)
        except Exception:
            pass
        logging.info("⏸️ paused")
        return True

    def _flush_final_result(self):
        self._finalized = True

    def finalize_and_get_results(self, wait: bool = True) -> dict:
        if wait:
            try: self.audio_queue.join()
            except Exception: pass
        t = self._processing_thread
        if t and t.is_alive():
            try: t.join(timeout=1.0)
            except Exception: pass
        self._flush_final_result()
        self.end_time_epoch = time.time()
        if not self.end_time_shamsi:
            self.end_time_shamsi = jdatetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
        total = self.total_count
        total_duration = self.current_elapsed_seconds(now=True)
        avg_between = (total_duration / total) if total > 0 and total_duration > 0 else None
        return {
            "total_count": total,
            "number": self.number or 0,
            "barnum": self.barnum or "",
            "start_time_shamsi": self.start_time_shamsi,
            "end_time_shamsi": self.end_time_shamsi,
            "start_time_epoch": self.start_time_epoch,
            "end_time_epoch": self.end_time_epoch,
            "average_time_between_words": avg_between,
        }

    def _record_loop(self):
        try:
            buf = bytearray(self._buffer_size)
            while self._record_evt.is_set():
                read_bytes = self._stream.read(buf, 0, len(buf))
                if read_bytes <= 0:
                    continue
                self.audio_queue.put([bytes(buf[:read_bytes])])
        except Exception as e:
            logging.error("record loop error: %s", e)
        finally:
            try:
                self._stream.stop(); self._stream.release()
            except Exception:
                pass
            try:
                if self._fx:
                    for v in self._fx.values():
                        try: v.release()
                        except Exception: pass
                    self._fx = {}
            except Exception:
                pass
            if isinstance(RN, _RnWrapper):
                RN.destroy()
            self._stream = None

    def _process_loop(self):
        STEP = 160  # 10 ms @ 16 kHz
        while self._record_evt.is_set() or not self.audio_queue.empty():
            try:
                frames = self.audio_queue.get(timeout=1)
            except Empty:
                continue
            try:
                raw = b"".join(frames)
                xh = array('h', raw)

                # prepend leftover
                if self._residual:
                    self._residual.extend(xh)
                    xh = self._residual
                    self._residual = array('h')

                out16 = array('h')
                use_rn  = isinstance(RN, _RnWrapper) and RN.st
                use_apm = False  # RN first; flip to True if you want to prefer APM

                for i in range(0, len(xh) - (len(xh) % STEP), STEP):
                    chunk = xh[i:i+STEP]
                    if use_rn:
                        y = RN.process_10ms_16k(chunk)
                        # light HPF to tame residual rumble
                        self._hpf.process_inplace(y)
                        out16.extend(y)
                    elif use_apm:
                        self._ensure_apm()
                        if self._apm:
                            out16.extend(self._apm.process(list(chunk)))
                        else:
                            self._pre.process_inplace(chunk); self._hpf.process_inplace(chunk); self._lpf.process_inplace(chunk)
                            out16.extend(chunk)
                    else:
                        self._pre.process_inplace(chunk); self._hpf.process_inplace(chunk); self._lpf.process_inplace(chunk)
                        out16.extend(chunk)

                # keep remainder for next time
                rem = len(xh) % STEP
                if rem:
                    self._residual = array('h', xh[-rem:])

                # feed Vosk
                self._ensure_recognizer()
                processed = out16.tobytes()
                if self._recognizer.AcceptWaveform(processed):
                    self._process_result_json(self._recognizer.Result())
                else:
                    self._process_result_json(self._recognizer.PartialResult())

            except Exception as e:
                logging.error("process loop: %s", e)
            finally:
                self.audio_queue.task_done()

    def _process_result_json(self, result_json: str):
        if not result_json: return
        try:
            result = json.loads(result_json)
        except Exception:
            return

        words_list = result.get("result", [])
        if words_list:
            i = 0
            while i < len(words_list):
                w = words_list[i]
                txt   = _normalize_fa(w.get("word", ""))
                conf  = float(w.get("conf", 1.0))
                end   = float(w.get("end",   0.0))
                if conf < MIN_CONF:
                    i += 1; continue
                if txt == "بسم" and i + 1 < len(words_list):
                    w2 = words_list[i + 1]
                    txt2  = _normalize_fa(w2.get("word", ""))
                    conf2 = float(w2.get("conf", 1.0))
                    gap   = float(w2.get("start", 0.0)) - float(w.get("end", 0.0))
                    if txt2 == "الله" and conf2 >= MIN_CONF and gap <= MERGE_GAP_S:
                        if (self._last_hit_time is None) or (end - self._last_hit_time >= DEBOUNCE_S):
                            self._bump_count(end)
                        i += 2; continue
                if txt in TARGETS:
                    if (self._last_hit_time is None) or (end - self._last_hit_time >= DEBOUNCE_S):
                        self._bump_count(end)
                i += 1
            return

        text = _normalize_fa(result.get("text") or result.get("partial") or "")
        if not text: return
        used = set()
        for word in TARGETS:
            for m in re.finditer(re.escape(word), text, flags=re.UNICODE):
                s, e = m.span()
                if all(pos not in used for pos in range(s, e)):
                    self.total_count += 1
                    used.update(range(s, e))
                    self._notify_count()

    def _bump_count(self, end_time: float):
        self.total_count += 1
        self._last_hit_time = end_time
        self._notify_count()

    def _notify_count(self):
        if self.on_count_update:
            try: self.on_count_update(self.total_count)
            except Exception: pass

    def current_elapsed_seconds(self, now: bool = False) -> int:
        if self.start_time_epoch <= 0: return 0
        end_ts = time.time() if now or self._record_evt.is_set() else (self.end_time_epoch or time.time())
        paused = self._pause_accum + (time.time() - self._pause_started if self._paused and self._pause_started else 0.0)
        return max(0, int(end_ts - self.start_time_epoch - paused))

"""
# write it to your project
# PROJECT = "/content/drive/MyDrive/voiceapp/project"
# os.makedirs(PROJECT, exist_ok=True)
# with open(os.path.join(PROJECT, "audio_engine.py"), "w", encoding="utf-8") as f:
#     f.write(audio_engine_py)
# print("✅ wrote audio_engine.py")

# ---------- buildozer.spec (persist caches on Drive; build in /content)
buildozer_spec = r'''
[app]
title = Voice Recognition App
package.name = voiceapp
package.domain = org.saria
version = 0.1.0

source.dir = .
source.main = main.py
source.include_exts = py,kv,png,jpg,json,zip

# Python + Kivy + JNI + Vosk (Python wrapper). APM removed.
requirements = python3==3.10, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime

# Native libs shipped with the app (RNNoise + Vosk .so in libs/arm64-v8a/)
android.add_libs_arm64_v8a = libs/arm64-v8a/*.so

# Bundle the Vosk model (your symlink points here)
android.add_assets = models

# Android toolchain
android.api = 33
android.minapi = 21
android.sdk = 33
android.ndk = 25b
android.ndk_api = 21

# Persist SDK/NDK caches on Drive
android.sdk_path = /content/drive/MyDrive/voiceapp/cache/android-sdk
android.ndk_path = /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b

# Permissions
android.permissions = RECORD_AUDIO, INTERNET, CAMERA, WRITE_EXTERNAL_STORAGE

# Target arch
android.archs = arm64-v8a

# (Removed as we wont be building APM right now)
# android.add_src = src
# p4a.local_recipes = ./recipes
# p4a.branch = develop
'''.strip()

# ---------- Java bridge
apm_java = r'''
package org.saria.apm;

public class ApmBridge {
    static { System.loadLibrary("apmbridge"); }

    private long handle;
    private final int sampleRate;

    // nsLevel: 0=LOW, 1=MODERATE, 2=HIGH, 3=VERY_HIGH
    public ApmBridge(boolean ns, int nsLevel, boolean agc, boolean hpf, boolean aec, int sampleRate) {
        this.sampleRate = sampleRate;
        this.handle = nativeCreate(ns, nsLevel, agc, hpf, aec, sampleRate);
        if (this.handle == 0) throw new RuntimeException("APM create failed");
    }

    // in: 10ms mono frame (160 samples at 16 kHz). Returns processed frame.
    public short[] process(short[] in) {
        if (handle == 0) return in;
        return nativeProcess(handle, in);
    }

    public void close() {
        if (handle != 0) { nativeDestroy(handle); handle = 0; }
    }

    private static native long nativeCreate(boolean ns, int nsLevel, boolean agc, boolean hpf, boolean aec, int sampleRate);
    private static native short[] nativeProcess(long handle, short[] in);
    private static native void nativeDestroy(long handle);
}
'''.strip()

# ---------- JNI shim
apm_jni = r'''
#include <jni.h>
#include <memory>
#include <vector>
#include <stdint.h>
#include <algorithm>

#include "modules/audio_processing/include/audio_processing.h"
#include "api/audio/audio_frame.h"

using webrtc::AudioProcessing;

struct APMCtx {
    std::unique_ptr<AudioProcessing> apm;
    int sample_rate = 16000;
    int channels = 1;
};

extern "C" {

static webrtc::AudioProcessing::Config::NoiseSuppression::Level ns_level_from_int(int v) {
    using L = webrtc::AudioProcessing::Config::NoiseSuppression::Level;
    switch(v){ case 3: return L::kVeryHigh; case 2: return L::kHigh; case 1: return L::kModerate; default: return L::kLow; }
}

JNIEXPORT jlong JNICALL
Java_org_saria_apm_ApmBridge_nativeCreate(JNIEnv* env, jclass,
    jboolean ns, jint nsLevel, jboolean agc, jboolean hpf, jboolean aec, jint sampleRate) {

    auto ctx = new APMCtx();
    ctx->sample_rate = sampleRate;

    webrtc::AudioProcessing::Config cfg;
    cfg.high_pass_filter.enabled = (bool)hpf;
    cfg.noise_suppression.enabled = (bool)ns;
    cfg.noise_suppression.level = ns_level_from_int(nsLevel);
    cfg.gain_controller2.enabled = (bool)agc;
    cfg.echo_canceller.enabled = (bool)aec;

    ctx->apm = webrtc::AudioProcessingBuilder().Create();
    if (!ctx->apm) { delete ctx; return 0; }
    ctx->apm->ApplyConfig(cfg);

    return reinterpret_cast<jlong>(ctx);
}

JNIEXPORT jshortArray JNICALL
Java_org_saria_apm_ApmBridge_nativeProcess(JNIEnv* env, jclass, jlong handle, jshortArray in_) {
    auto* ctx = reinterpret_cast<APMCtx*>(handle);
    if (!ctx || !ctx->apm) return in_;

    jsize n = env->GetArrayLength(in_);
    std::vector<int16_t> in_buf(n);
    env->GetShortArrayRegion(in_, 0, n, reinterpret_cast<jshort*>(in_buf.data()));

    // Convert to float [-1,1]
    std::vector<float> fbuf(n);
    for (int i=0;i<n;++i) fbuf[i] = static_cast<float>(in_buf[i]) / 32768.0f;

    webrtc::StreamConfig cfg(ctx->sample_rate, ctx->channels);
    float* ch_in[1]  = { fbuf.data() };
    float* ch_out[1] = { fbuf.data() };

    if (ctx->apm->ProcessStream(ch_out, cfg, cfg, ch_out) != webrtc::AudioProcessing::kNoError) {
        return in_;
    }

    // Convert back to int16
    std::vector<int16_t> out_buf(n);
    for (int i=0;i<n;++i) {
        float x = fbuf[i] * 32768.0f;
        if (x > 32767.f) x = 32767.f; else if (x < -32768.f) x = -32768.f;
        out_buf[i] = static_cast<int16_t>(x);
    }
    jshortArray out_ = env->NewShortArray(n);
    env->SetShortArrayRegion(out_, 0, n, reinterpret_cast<jshort*>(out_buf.data()));
    return out_;
}

JNIEXPORT void JNICALL
Java_org_saria_apm_ApmBridge_nativeDestroy(JNIEnv*, jclass, jlong handle) {
    auto* ctx = reinterpret_cast<APMCtx*>(handle);
    delete ctx;
}

} // extern "C"
'''.strip()

# ---------- CMakeLists for JNI (prefers prebuilt lib; otherwise you must add sources)
cmakelists = r'''
cmake_minimum_required(VERSION 3.10)
project(apmbridge)

add_library(apmbridge SHARED
    apm_jni.cpp
)

# Include dirs for vendored WebRTC APM (adjust if needed)
include_directories(
    ${CMAKE_SOURCE_DIR}/../../third_party/webrtc-apm/include
    ${CMAKE_SOURCE_DIR}/../../third_party/webrtc-apm/modules
    ${CMAKE_SOURCE_DIR}/../../third_party/webrtc-apm
)

# --- Option A: link a prebuilt static lib (RECOMMENDED) ---
# Expect: third_party/webrtc-apm/arm64-v8a/libwebrtc_apm.a
add_library(webrtc_apm STATIC IMPORTED)
set_target_properties(webrtc_apm PROPERTIES
  IMPORTED_LOCATION ${CMAKE_SOURCE_DIR}/../../third_party/webrtc-apm/arm64-v8a/libwebrtc_apm.a
)

target_link_libraries(apmbridge webrtc_apm log)
'''.strip()

# ---------- p4a local recipe
recipe_py = r'''
from pythonforandroid.recipe import CompiledComponentsPythonRecipe
from pythonforandroid.util import current_directory
import os

class ApmBridgeRecipe(CompiledComponentsPythonRecipe):
    version = 'local'
    name = 'apmbridge'
    built_libraries = {'libapmbridge.so': '.'}

    def build_arch(self, arch):
        super().build_arch(arch)
        build_dir = os.path.join(self.get_build_dir(arch), 'build')
        os.makedirs(build_dir, exist_ok=True)
        src_dir = os.path.join(self.get_build_dir(arch))
        with current_directory(build_dir):
            self.ctx.cmd(['cmake',
                          f'-DANDROID_ABI={arch.arch}',
                          f'-DANDROID_PLATFORM=android-{self.ctx.ndk_api}',
                          f'-DANDROID_STL=c++_shared',
                          f'-DCMAKE_TOOLCHAIN_FILE={self.ctx.ndk_dir}/build/cmake/android.toolchain.cmake',
                          src_dir + '/jni/apm'])
            self.ctx.cmd(['cmake', '--build', '.', '--config', 'Release'])
            self.copy_build_libraries(arch)

recipe = ApmBridgeRecipe()
'''.strip()

# Write files
pathlib.Path(f"{ROOT}/main.py").write_text(main_py, encoding="utf-8")
pathlib.Path(f"{ROOT}/audio_engine.py").write_text(audio_engine_py, encoding="utf-8")
pathlib.Path(f"{ROOT}/buildozer.spec").write_text(buildozer_spec, encoding="utf-8")

# Java / JNI / recipe
os.makedirs(f"{ROOT}/src/org/saria/apm", exist_ok=True)
os.makedirs(f"{ROOT}/jni/apm", exist_ok=True)
os.makedirs(f"{ROOT}/recipes/apmbridge", exist_ok=True)

pathlib.Path(f"{ROOT}/src/org/saria/apm/ApmBridge.java").write_text(apm_java, encoding="utf-8")
pathlib.Path(f"{ROOT}/jni/apm/apm_jni.cpp").write_text(apm_jni, encoding="utf-8")
pathlib.Path(f"{ROOT}/jni/apm/CMakeLists.txt").write_text(cmakelists, encoding="utf-8")
pathlib.Path(f"{ROOT}/recipes/apmbridge/recipe.py").write_text(recipe_py, encoding="utf-8")

print("✅ Wrote updated sources & build files into", ROOT)


✅ Wrote updated sources & build files into /content/drive/MyDrive/voiceapp/project


In [ ]:
!ls

drive  sample_data  venv310


In [ ]:
%%bash
set -e
cd /content/drive/MyDrive/voiceapp/project

# 1) Ensure a version in [app]
if ! grep -qE '^[[:space:]]*version[[:space:]]*=' buildozer.spec; then
  # insert just after the [app] header
  sed -i '/^\[app\]/a version = 0.1.0' buildozer.spec
fi

# 2) Requirements (RNNoise is a .so we bundle, Vosk is pure-Python and loads libvosk.so)
sed -i 's|^requirements *=.*|requirements = python3==3.10, kivy==2.3.0, pyjnius, jdatetime, vosk==0.3.45|' buildozer.spec || true

# 3) Make sure we load bundled .so files (RNNoise + Vosk native lib)
grep -q '^android.add_libs_arm64_v8a' buildozer.spec || echo 'android.add_libs_arm64_v8a = libs/arm64-v8a/*.so' >> buildozer.spec

# 4) Keep assets + arch tidy (add if missing)
grep -q '^android.add_assets' buildozer.spec || echo 'android.add_assets = models' >> buildozer.spec
grep -q '^android.archs' buildozer.spec || echo 'android.archs = arm64-v8a' >> buildozer.spec

echo "----- buildozer.spec (relevant lines) -----"
grep -E '^\[app\]|^version *=|^requirements *=|^android.add_libs_arm64_v8a|^android.add_assets|^android.archs' -n buildozer.spec


----- buildozer.spec (relevant lines) -----
1:[app]
2:version = 0.1.0
13:requirements = python3==3.10, kivy==2.3.0, pyjnius, jdatetime, vosk==0.3.45
16:android.add_libs_arm64_v8a = libs/arm64-v8a/*.so
19:android.add_assets = models
41:android.archs = arm64-v8a


In [ ]:
%%bash
cd /content/drive/MyDrive/voiceapp/project
mkdir -p libs/arm64-v8a
# assuming you placed the AAR as vosk-android.aar in the project dir:
unzip -j vosk-android.aar 'jni/arm64-v8a/libvosk.so' -d libs/arm64-v8a
ls -l libs/arm64-v8a/


total 5597
-rw------- 1 root root 5730728 Nov  1 08:18 librnnoise.so


unzip:  cannot find or open vosk-android.aar, vosk-android.aar.zip or vosk-android.aar.ZIP.


In [ ]:
%%bash
cd /content/drive/MyDrive/voiceapp/project
test -f libs/arm64-v8a/librnnoise.so || echo "⚠️ missing RNNoise: libs/arm64-v8a/librnnoise.so"
test -f libs/arm64-v8a/libvosk.so   || echo "⚠️ missing Vosk native: libs/arm64-v8a/libvosk.so"


⚠️ missing Vosk native: libs/arm64-v8a/libvosk.so


In [ ]:
# 5
# PROJECT_ROOT="/content/drive/MyDrive/voiceapp/project"
# CACHE_ROOT="/content/drive/MyDrive/voiceapp/cache"

# rm -rf "$PROJECT_ROOT/models"
# ln -s "$CACHE_ROOT/models-fa" "$PROJECT_ROOT/models"

# echo "Models symlink:"
# ls -l "$PROJECT_ROOT/models"


# CACHE_ROOT="/content/drive/MyDrive/voiceapp/cache"
# mkdir -p "$CACHE_ROOT/models-fa"
# cd "$CACHE_ROOT/models-fa"

# if [ ! -d "vosk-model-fa-0.42" ]; then
#   echo "Downloading Vosk FA 0.42 into Drive cache..."
#   wget -q https://alphacephei.com/vosk/models/vosk-model-fa-0.42.zip -O vosk-model-fa-0.42.zip
#   unzip -q vosk-model-fa-0.42.zip
#   rm -f vosk-model-fa-0.42.zip
# fi

# # Project link
# PROJECT_ROOT="/content/drive/MyDrive/voiceapp/project"
# rm -rf "$PROJECT_ROOT/models"
# ln -s "$CACHE_ROOT/models-fa/vosk-model-fa-0.42" "$PROJECT_ROOT/models"

# echo "Models at:"; ls -l "$PROJECT_ROOT/models"

# Fast download to /content (session only)


%%bash
set -e

# Install tools once
sudo apt-get update -y
sudo apt-get install -y aria2 unzip zstd

MODEL_URL="https://alphacephei.com/vosk/models/vosk-model-fa-0.42.zip"
LOCAL_ROOT="/content/models-fa"
mkdir -p "$LOCAL_ROOT"
cd "$LOCAL_ROOT"

# Fast multi-connection download into /content (ephemeral but speedy)
if [ ! -d "vosk-model-fa-0.42" ]; then
  echo "▶ downloading model to /content (multi-connection)…"
  aria2c -x 16 -s 16 --continue=true -o vosk-model-fa-0.42.zip "$MODEL_URL"
  unzip -q -o vosk-model-fa-0.42.zip
  rm -f vosk-model-fa-0.42.zip
fi

# Link into your Drive project (make sure Drive is mounted)
PROJECT_ROOT="/content/drive/MyDrive/voiceapp/project"
mkdir -p "$PROJECT_ROOT"
rm -rf "$PROJECT_ROOT/models"
ln -s "$LOCAL_ROOT/vosk-model-fa-0.42" "$PROJECT_ROOT/models"

echo "Models link ->"
ls -l "$PROJECT_ROOT/models"


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-26ubuntu3.2).
The following packages were automatically installed and are no longer r

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 4.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
!ls -l /content/drive/MyDrive/voiceapp/project/libs/arm64-v8a/librnnoise.so

-rw------- 1 root root 5730728 Nov  1 08:18 /content/drive/MyDrive/voiceapp/project/libs/arm64-v8a/librnnoise.so


In [ ]:
VENV = "/content/venv310"

# Make sure core tools are up-to-date in the venv
!{VENV}/bin/pip install -U pip setuptools "wheel~=0.43.0"

# Preinstall the host deps Buildozer tries to install with --user
!{VENV}/bin/pip install -U appdirs "colorama>=0.3.3" jinja2 'sh>=2,<3.0' meson ninja build toml packaging


  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.1
    Uninstalling wheel-0.45.1:
      Successfully uninstalled wheel-0.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [build]


In [ ]:
# --- Install Python 3.10 + pip ---
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-venv python3.10-distutils

# Ensure pip exists for 3.10 and is current
!python3.10 -m ensurepip --upgrade
!python3.10 -m pip install --upgrade pip

# --- Install Buildozer & host deps into the *user* site ---
!python3.10 -m pip install --user "cython==0.29.36" buildozer \
  appdirs "colorama>=0.3.3" jinja2 "sh>=2,<3.0" meson ninja build toml packaging setuptools "wheel~=0.43.0"

# Put the user bin on PATH for this notebook process
import os
os.environ["PATH"] += ":/root/.local/bin"

# Sanity
!which buildozer || true
!buildozer --version
!python3.10 -V


# Important: From now on, do not call {VENV}/bin/buildozer. Use !buildozer … (or explicitly !/root/.local/bin/buildozer …).

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
%%bash
# Don't fail the whole cell on first hiccup
set -u

echo "== Prep tools =="
sudo apt-get update -y || true
sudo apt-get install -y curl unzip || true

echo "== Move to project =="
cd /content/drive/MyDrive/voiceapp/project || { echo "Project path missing"; exit 1; }
mkdir -p libs/arm64-v8a

AAR="vosk-android-0.3.45.aar"

# If file already exists from a previous run, keep it
if [ ! -s "$AAR" ]; then
  echo "== Try downloading AAR from Maven =="
  urls=(
    "https://repo1.maven.org/maven2/com/alphacephei/vosk-android/0.3.45/vosk-android-0.3.45.aar"
    "https://repo.maven.apache.org/maven2/com/alphacephei/vosk-android/0.3.45/vosk-android-0.3.45.aar"
    "https://repo1.maven.org/maven2/org/alphacephei/vosk-android/0.3.45/vosk-android-0.3.45.aar"
    "https://repo1.maven.org/maven2/org/vosk/android/0.3.45/android-0.3.45.aar"
  )
  got=""
  for u in "${urls[@]}"; do
    echo "-> $u"
    # Retry a few times, keep verbose so you see what happens
    if curl -fSL --retry 5 --retry-delay 2 --retry-all-errors -o "$AAR" "$u"; then
      got="yes"; break
    fi
  done
  if [ -z "$got" ]; then
    echo "❌ Could not download the AAR automatically (network/mirror issue)."
    echo "   Use the MANUAL UPLOAD cell below instead."
    exit 0
  fi
else
  echo "== Using existing $AAR =="
fi

echo "== List arm64 contents =="
unzip -l "$AAR" | grep -E 'jni/arm64-v8a/.*\.so' || echo "(no listing found; continuing)"

echo "== Extract libvosk.so to libs/arm64-v8a =="
# -o overwrite in case you re-run
unzip -oj "$AAR" 'jni/arm64-v8a/*.so' -d libs/arm64-v8a || { echo "Unzip failed"; exit 1; }

echo "== Result =="
ls -lh libs/arm64-v8a || true


== Prep tools ==
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
unzip is already the newest version (6.0-26ubuntu3.2

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%bash
set -u
cd /content/drive/MyDrive/voiceapp/project || exit 1
echo "== libs/arm64-v8a =="
ls -lh libs/arm64-v8a || true
echo
echo "== models symlink =="
ls -ld models || echo "models symlink missing (make the /content/models-fa/vosk-model-fa-0.42 link)"


== libs/arm64-v8a ==
total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so

== models symlink ==
lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42


1) Install Buildozer (no venv; avoid the --user in virtualenv trap)

In [ ]:
# Colab Python cell
%pip install -q --upgrade pip wheel setuptools
%pip install -q --no-warn-script-location "cython==0.29.36" buildozer

# Sanity: show where it landed and its version
!which buildozer || true
!python -m buildozer --version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
/usr/local/bin/buildozer
# Check configuration tokens
Buildozer 1.5.0


Set paths and sanity print

In [ ]:
import os, pathlib, shutil, subprocess, sys
ANDROIDSDK = "/content/drive/MyDrive/voiceapp/cache/android-sdk"
ANDROIDNDK = "/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"

os.makedirs(ANDROIDSDK, exist_ok=True)
os.makedirs(ANDROIDNDK, exist_ok=True)

os.environ["ANDROID_HOME"] = ANDROIDSDK
os.environ["ANDROID_SDK_ROOT"] = ANDROIDSDK
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

print("ANDROID_HOME =", os.environ["ANDROID_HOME"])
print("ANDROID_NDK_HOME target (to create) =", ANDROIDNDK)


ANDROID_HOME = /content/drive/MyDrive/voiceapp/cache/android-sdk
ANDROID_NDK_HOME target (to create) = /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b


Install JDK and common tools (no output suppression)

In [ ]:
%%bash
set -euxo pipefail
sudo apt-get update -y
sudo apt-get install -y openjdk-17-jdk unzip curl
java -version


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
unzip is already the newest version (6.0-26ubuntu3.2).
openjdk-17-jdk

+ sudo apt-get update -y
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sudo apt-get install -y openjdk-17-jdk unzip curl
+ java -version
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


Install Android cmdline-tools, accept licenses, and install API 33 + build-tools

In [ ]:
%%bash
set -euo pipefail
set -x
trap 'echo "❌ Failed at line $LINENO"; exit 1' ERR

ANDROIDSDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"

# --- Ensure Java + tools ---
if ! command -v java >/dev/null 2>&1; then
  sudo apt-get update -y
  sudo apt-get install -y openjdk-17-jdk curl unzip
fi
java -version

# --- Download cmdline-tools into .../cmdline-tools/latest/ if missing ---
if [ ! -x "${ANDROIDSDK}/cmdline-tools/latest/bin/sdkmanager" ]; then
  mkdir -p "${ANDROIDSDK}/cmdline-tools"
  TMP="$(mktemp -d)"
  cd "$TMP"
  curl -fL -o tools.zip "https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip"
  unzip -q tools.zip
  rm -f tools.zip
  mkdir -p "${ANDROIDSDK}/cmdline-tools/latest"

  # The zip contains a top-level "cmdline-tools" folder
  if [ -d cmdline-tools/bin ]; then
    mv cmdline-tools/* "${ANDROIDSDK}/cmdline-tools/latest/"
  else
    echo "Unexpected layout after unzip. Contents:"
    find .
    exit 1
  fi

  cd /
  rm -rf "$TMP"
fi

export ANDROID_SDK_ROOT="${ANDROIDSDK}"
export ANDROID_HOME="${ANDROIDSDK}"

# --- Show sdkmanager version ---
"${ANDROIDSDK}/cmdline-tools/latest/bin/sdkmanager" --sdk_root="${ANDROIDSDK}" --version

# --- Accept licenses (non-interactive) ---
yes | "${ANDROIDSDK}/cmdline-tools/latest/bin/sdkmanager" --sdk_root="${ANDROIDSDK}" --licenses || true

# --- Install platform + build-tools (API 33) ---
"${ANDROIDSDK}/cmdline-tools/latest/bin/sdkmanager" --sdk_root="${ANDROIDSDK}" \
  "platforms;android-33" "build-tools;33.0.2"

# --- List a few installed packages for sanity ---
"${ANDROIDSDK}/cmdline-tools/latest/bin/sdkmanager" --sdk_root="${ANDROIDSDK}" --list | head -n 50 || true

echo "✅ SDK ready at: ${ANDROIDSDK}"


12.0

[=======================================] 100% Computing updates...             
All SDK package licenses accepted.

[=======================================] 100% Unzipping... android-13/framework

[=======================================] 100% Computing updates...             
Installed packages:
  Path                 | Version | Description                    | Location            
  -------              | ------- | -------                        | -------             
  build-tools;33.0.2   | 33.0.2  | Android SDK Build-Tools 33.0.2 | build-tools/33.0.2  
  platforms;android-33 | 3       | Android SDK Platform 33        | platforms/android-33

Available Packages:
  Path                                                                            | Version           | Description                                                           
  -------                                                                         | -------           | -------                               

+ trap 'echo "❌ Failed at line $LINENO"; exit 1' ERR
+ ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ command -v java
+ java -version
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)
+ '[' '!' -x /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager ']'
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/drive/MyDrive/voiceapp/cache/android-sdk --version
+ yes
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content

Install NDK r25b to Drive cache

In [ ]:
%%bash
set -euxo pipefail
ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"

if [ ! -e "${ANDROIDNDK}/source.properties" ]; then
  cd /content
  curl -L -o ndk.zip "https://dl.google.com/android/repository/android-ndk-r25b-linux.zip"
  unzip -q ndk.zip
  rm -f ndk.zip
  rm -rf "${ANDROIDNDK}"
  mv /content/android-ndk-r25b "${ANDROIDNDK}"
fi

echo "NDK present at: ${ANDROIDNDK}"
ls -l "${ANDROIDNDK}" | head


NDK present at: /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
total 1398
drwx------+ 6 root root   4096 Aug 12  2022 build
-rw-------+ 1 root root   5308 Aug 12  2022 CHANGELOG.md
drwx------+ 2 root root   4096 Aug 12  2022 meta
-rw-------+ 1 root root     73 Aug 12  2022 ndk-build
-rw-------+ 1 root root     91 Aug 12  2022 ndk-gdb
-rw-------+ 1 root root     91 Aug 12  2022 ndk-lldb
-rw-------+ 1 root root     93 Aug 12  2022 ndk-stack
-rw-------+ 1 root root     93 Aug 12  2022 ndk-which
-rw-------+ 1 root root 547910 Aug 12  2022 NOTICE


+ ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ '[' '!' -e /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b/source.properties ']'
+ cd /content
+ curl -L -o ndk.zip https://dl.google.com/android/repository/android-ndk-r25b-linux.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  506M  100  506M    0     0  20.7M      0  0:00:24  0:00:24 --:--:-- 22.1M
+ unzip -q ndk.zip
+ rm -f ndk.zip
+ rm -rf /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ mv /content/android-ndk-r25b /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ echo 'NDK present at: /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b'
+ ls -l /content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ head


SKDmanager

In [ ]:
%%bash
set -euxo pipefail

SDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"

# 1) Inspect what's there so we know why it failed before
ls -la "$SDK" || true
ls -la "$SDK/cmdline-tools" || true
ls -la "$SDK/cmdline-tools/latest/bin" || true

# 2) (Re)install cmdline-tools if sdkmanager isn't present yet
if [ ! -x "$SDK/cmdline-tools/latest/bin/sdkmanager" ]; then
  echo "Re-installing Android cmdline-tools..."
  sudo apt-get update -y >/dev/null
  sudo apt-get install -y openjdk-17-jdk curl unzip >/dev/null

  mkdir -p "$SDK/cmdline-tools"
  TMP="$(mktemp -d)"
  cd "$TMP"
  curl -fL -o tools.zip "https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip"
  unzip -q tools.zip
  rm -f tools.zip
  mkdir -p "$SDK/cmdline-tools/latest"
  # zip has a top-level folder "cmdline-tools"
  mv cmdline-tools/* "$SDK/cmdline-tools/latest/" || true
  cd /
  rm -rf "$TMP"
fi

# 3) Create legacy tools/bin WRAPPERS (scripts), more robust than symlinks on Drive
mkdir -p "$SDK/tools/bin"

cat >"$SDK/tools/bin/sdkmanager" <<'WRAP'
#!/usr/bin/env bash
# Resolve to .../cmdline-tools/latest/bin relative to this script
HERE="$(cd "$(dirname "$0")" && pwd)"
REAL="$(cd "$HERE/../../cmdline-tools/latest/bin" && pwd)/sdkmanager"
exec "$REAL" "$@"
WRAP
chmod +x "$SDK/tools/bin/sdkmanager"

if [ -x "$SDK/cmdline-tools/latest/bin/avdmanager" ]; then
cat >"$SDK/tools/bin/avdmanager" <<'WRAP'
#!/usr/bin/env bash
HERE="$(cd "$(dirname "$0")" && pwd)"
REAL="$(cd "$HERE/../../cmdline-tools/latest/bin" && pwd)/avdmanager"
exec "$REAL" "$@"
WRAP
chmod +x "$SDK/tools/bin/avdmanager"
fi

echo "Legacy wrappers created:"
ls -l "$SDK/tools/bin"

# 4) Prove both paths work
"$SDK/cmdline-tools/latest/bin/sdkmanager" --version
"$SDK/tools/bin/sdkmanager" --version


total 25
drwx------ 3 root root 4096 Nov  2 06:55 build-tools
drwx------ 4 root root 4096 Nov  2 06:54 cmdline-tools
-rw------- 1 root root   16 Nov  2 07:10 .knownPackages
drwx------ 2 root root 4096 Nov  2 06:54 licenses
drwx------ 3 root root 4096 Nov  2 07:10 platforms
drwx------ 2 root root 4096 Nov  2 07:10 .temp
drwx------ 3 root root 4096 Nov  2 07:19 tools
total 8
drwx------ 4 root root 4096 Nov  2 06:54 cmdline-tools
drwx------ 4 root root 4096 Nov  2 07:05 latest
total 44
-rwx------ 1 root root 5321 Jan  1  2010 apkanalyzer
-rwx------ 1 root root 5312 Jan  1  2010 avdmanager
-rwx------ 1 root root 5280 Jan  1  2010 lint
-rwx------ 1 root root 5253 Jan  1  2010 profgen
-rwx------ 1 root root 5308 Jan  1  2010 resourceshrinker
-rwx------ 1 root root 5253 Jan  1  2010 retrace
-rwx------ 1 root root 5309 Jan  1  2010 screenshot2
-rwx------ 1 root root 5319 Jan  1  2010 sdkmanager
Legacy wrappers created:
total 1
-rwx------ 1 root root 146 Nov  2 07:23 avdmanager
-rwx------ 1 roo

+ SDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ls -la /content/drive/MyDrive/voiceapp/cache/android-sdk
+ ls -la /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools
+ ls -la /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin
+ '[' '!' -x /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager ']'
+ mkdir -p /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin
+ cat
+ chmod +x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager
+ '[' -x /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/avdmanager ']'
+ cat
+ chmod +x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/avdmanager
+ echo 'Legacy wrappers created:'
+ ls -l /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager --version
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --vers

Quick preflight (sanity check)

In [ ]:
%%bash
set -euo pipefail
cd /content/drive/MyDrive/voiceapp/project

echo "== libs present =="
ls -lh libs/arm64-v8a

echo "== model link =="
ls -ld models


== libs present ==
total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so
== model link ==
lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42


In [ ]:
# Patch buildozer.spec to use a real Python tarball (3.10.13)
import re, pathlib
spec = pathlib.Path("/content/drive/MyDrive/voiceapp/project/buildozer.spec")
txt = spec.read_text(encoding="utf-8")
txt = re.sub(r'python3==\s*3\.10\b', 'python3==3.10.13', txt)  # only change if it's "3.10"
spec.write_text(txt, encoding="utf-8")
print("✅ buildozer.spec updated to python3==3.10.13")

# Optional: nuke any partial python3 downloads so p4a refetches cleanly
import shutil, os
base = "/content/drive/MyDrive/voiceapp/project/.buildozer/android/platform"
for p in [
    f"{base}/build-arm64-v8a/packages/python3",
    f"{base}/build-arm64-v8a/packages/python3*",
]:
    shutil.rmtree(p, ignore_errors=True)
print("🧹 cleaned old python3 package cache (if any🤷‍♂️)")


✅ buildozer.spec updated to python3==3.10.13
🧹 cleaned old python3 package cache (if any)


Patch python version in buildozer.spec(keep 3.10 but real tarball

In [ ]:
# Patch python3==3.10 -> 3.10.13 (the tarball that exists)
import re, pathlib
spec = pathlib.Path("/content/drive/MyDrive/voiceapp/project/buildozer.spec")
txt = spec.read_text(encoding="utf-8")
txt = re.sub(r'python3==\s*3\.10\b', 'python3==3.10.13', txt)
spec.write_text(txt, encoding="utf-8")
print("✅ buildozer.spec now uses python3==3.10.13")


✅ buildozer.spec now uses python3==3.10.13


Move the build workdir off Drive (symlink .buildozer → /content/.buildozer)

Build the damn app

In [ ]:
# Colab Python cell
import os
%cd /content/drive/MyDrive/voiceapp/project

# point Buildozer/Gradle/SDK/NDK to your Drive cache
os.environ['GRADLE_USER_HOME'] = '/content/drive/MyDrive/voiceapp/cache/.gradle'
os.environ['ANDROID_HOME']     = '/content/drive/MyDrive/voiceapp/cache/android-sdk'
os.environ['ANDROIDSDK']       = '/content/drive/MyDrive/voiceapp/cache/android-sdk'
os.environ['ANDROIDNDK']       = '/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b'
os.environ['JAVA_HOME']        = '/usr/lib/jvm/java-17-openjdk-amd64'

# run the global buildozer (installed at /usr/local/bin)
!python -m buildozer -v android debug --storage-dir=/content/.buildozer --arch=arm64-v8a


/content/drive/MyDrive/voiceapp/project
# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
# WARNING: Config token app android.sdk is deprecated and ignored, but you set value 33
# Ensure build layout
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> found at /usr/local/bin/cython
# Search for Java compiler (javac)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/javac
# Search for Java keytool (keytool)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool
# Install platform
# Run ['git', 'config', '--get', 'remote.origin.url']
# Cwd /content/drive/MyDrive/voiceapp/project/.buildozer/android/platform/python-for-android
https://github.com/kivy/python-for-android.git
# Run ['git', 'branch', '-vv']
# Cwd /content/drive/MyDrive/voiceapp/project/.buildozer/

In [ ]:
!sed -n '1,200p' /content/.buildozer/android/platform/build-arm64-v8a/build/other_builds/libffi/*/libffi/config.log | sed -n '/configure: error/,$p'


sed: can't read /content/.buildozer/android/platform/build-arm64-v8a/build/other_builds/libffi/*/libffi/config.log: No such file or directory


In [ ]:
from google.colab import files
uploaded = files.upload()  # pick vosk-android-0.3.45.aar or android-0.3.45.aar

Saving librnnoise.so to librnnoise.so


s

In [ ]:
# Colab Python cell
import os, shutil, subprocess, pathlib

PROJ = "/content/drive/MyDrive/voiceapp/project"
LOCAL = "/content/.buildozer"
os.makedirs(LOCAL, exist_ok=True)

p = pathlib.Path(PROJ) / ".buildozer"
if p.exists() or p.is_symlink():
    try:
        if p.is_symlink():
            p.unlink()
        else:
            shutil.rmtree(p)
        print("🧹 removed project/.buildozer on Drive")
    except Exception as e:
        print("⚠️ could not remove:", e)

os.symlink(LOCAL, p)
print("🔗 linked", p, "->", LOCAL)

subprocess.run(["bash","-lc",f"ls -ld '{p}' && ls -ld '{LOCAL}'"], check=False)


🧹 removed project/.buildozer on Drive
🔗 linked /content/drive/MyDrive/voiceapp/project/.buildozer -> /content/.buildozer


CompletedProcess(args=['bash', '-lc', "ls -ld '/content/drive/MyDrive/voiceapp/project/.buildozer' && ls -ld '/content/.buildozer'"], returncode=0)

In [ ]:
# Colab Python cell
import os
os.environ['GRADLE_USER_HOME'] = '/content/drive/MyDrive/voiceapp/cache/.gradle'
os.environ['ANDROID_HOME']     = '/content/drive/MyDrive/voiceapp/cache/android-sdk'
os.environ['ANDROIDSDK']       = os.environ['ANDROID_HOME']
os.environ['ANDROIDNDK']       = '/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b'
os.environ['JAVA_HOME']        = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH']            += f":{os.environ['ANDROID_HOME']}/tools/bin:/root/.local/bin"

!sudo apt-get update -y >/dev/null
!sudo apt-get install -y autoconf automake libtool pkg-config >/dev/null

!mkdir -p "$ANDROID_HOME/tools/bin"
!ln -sf "$ANDROID_HOME/cmdline-tools/latest/bin/sdkmanager" "$ANDROID_HOME/tools/bin/sdkmanager" || true
!ln -sf "$ANDROID_HOME/cmdline-tools/latest/bin/avdmanager" "$ANDROID_HOME/tools/bin/avdmanager" || true

!echo "ANDROID_HOME=$ANDROID_HOME"
!echo "ANDROIDNDK=$ANDROIDNDK"
!echo "GRADLE_USER_HOME=$GRADLE_USER_HOME"


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ln: failed to create symbolic link '/content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager': Operation not supported
ln: failed to create symbolic link '/content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/avdmanager': Operation not supported
ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle


In [ ]:
# Colab Python cell
import re, pathlib
spec = pathlib.Path("/content/drive/MyDrive/voiceapp/project/buildozer.spec")
txt = spec.read_text(encoding="utf-8")
txt = re.sub(r'python3==\s*3\.10\b', 'python3==3.10.13', txt)  # or switch to 3.11.9 if you like
spec.write_text(txt, encoding="utf-8")
print("✅ buildozer.spec now uses python3==3.10.13")


✅ buildozer.spec now uses python3==3.10.13


In [ ]:
%%bash
set -euxo pipefail
rm -rf /content/.buildozer/android/platform/build-arm64-v8a/build/other_builds/libffi || true
rm -rf /content/.buildozer/android/platform/build-arm64-v8a/packages/python3* || true
echo "🧨 cleared libffi & python3 caches under /content/.buildozer"


🧨 cleared libffi & python3 caches under /content/.buildozer


+ rm -rf /content/.buildozer/android/platform/build-arm64-v8a/build/other_builds/libffi
+ rm -rf '/content/.buildozer/android/platform/build-arm64-v8a/packages/python3*'
+ echo '🧨 cleared libffi & python3 caches under /content/.buildozer'


Fixing Enviroment error

In [ ]:
# Colab Python cell
import os, shutil, pathlib, subprocess
PROJ = "/content/drive/MyDrive/voiceapp/project"
LOCAL = "/content/.buildozer"
os.makedirs(LOCAL, exist_ok=True)

p = pathlib.Path(PROJ) / ".buildozer"
if p.exists() or p.is_symlink():
    try:
        if p.is_symlink():
            p.unlink()
        else:
            shutil.rmtree(p)
        print("🧹 removed project/.buildozer on Drive")
    except Exception as e:
        print("⚠️ could not remove:", e)

os.symlink(LOCAL, p)
print("🔗 linked", p, "->", LOCAL)
subprocess.run(["bash","-lc",f"ls -ld '{p}' && ls -ld '{LOCAL}'"], check=False)


🧹 removed project/.buildozer on Drive
🔗 linked /content/drive/MyDrive/voiceapp/project/.buildozer -> /content/.buildozer


CompletedProcess(args=['bash', '-lc', "ls -ld '/content/drive/MyDrive/voiceapp/project/.buildozer' && ls -ld '/content/.buildozer'"], returncode=0)

In [ ]:
%%bash
set -euxo pipefail
sudo apt-get update -y
sudo apt-get install -y build-essential autoconf automake libtool pkg-config \
                        zip unzip curl git ccache cmake


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
autoconf is already the newest version (2.71-2).
automake is already the newest version (1:1.16.5-1.3).
build-essential is alr

+ sudo apt-get update -y
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sudo apt-get install -y build-essential autoconf automake libtool pkg-config zip unzip curl git ccache cmake
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fix sdkmanager/avdmanager with wrappers (not copies)

In [ ]:
%%bash
set -euxo pipefail
SDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"

# Verify the legacy wrapper exists and works
test -x "${SDK}/tools/bin/sdkmanager"
"${SDK}/tools/bin/sdkmanager" --version

# Make sure the essentials are installed (some builds die if platform-tools is missing)
yes | "${SDK}/tools/bin/sdkmanager" --licenses || true
"${SDK}/tools/bin/sdkmanager" \
  "platform-tools" \
  "platforms;android-33" \
  "build-tools;33.0.2"

echo "Installed packages (head):"
"${SDK}/cmdline-tools/latest/bin/sdkmanager" --list | head -n 50 || true


12.0

[=======================================] 100% Computing updates...             
All SDK package licenses accepted.

[=======================================] 100% Computing updates...             

Installed packages (head):
[=======================================] 100% Computing updates...             
Installed packages:
  Path                 | Version | Description                    | Location            
  -------              | ------- | -------                        | -------             
  build-tools;33.0.2   | 33.0.2  | Android SDK Build-Tools 33.0.2 | build-tools/33.0.2  
  build-tools;36.1.0   | 36.1.0  | Android SDK Build-Tools 36.1   | build-tools/36.1.0  
  platform-tools       | 36.0.0  | Android SDK Platform-Tools     | platform-tools      
  platforms;android-33 | 3       | Android SDK Platform 33        | platforms/android-33

Available Packages:
  Path                                                                            | Version           | Descript

+ SDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ test -x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --version
+ yes
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --licenses
+ true
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager platform-tools 'platforms;android-33' 'build-tools;33.0.2'
+ echo 'Installed packages (head):'
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin/sdkmanager --list
+ head -n 50


In [ ]:
%%bash
set -euxo pipefail
rm -rf /content/.buildozer || true
rm -rf /content/drive/MyDrive/voiceapp/project/.buildozer || true


+ rm -rf /content/.buildozer
+ rm -rf /content/drive/MyDrive/voiceapp/project/.buildozer


In [ ]:
# Colab Python cell
import re, pathlib
spec = pathlib.Path("/content/drive/MyDrive/voiceapp/project/buildozer.spec")
txt  = spec.read_text(encoding="utf-8")
txt = re.sub(r'(?m)^(requirements\s*=).*$',
             r'\1 python3==3.10.13, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime',
             txt)
spec.write_text(txt, encoding="utf-8")
print([l for l in txt.splitlines() if l.strip().startswith("requirements")][0])


requirements = python3==3.10.13, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime


In [ ]:
%%bash
# Do NOT use -e here; we want to keep running even if p4a fails.
set -uox pipefail
cd /content/drive/MyDrive/voiceapp/project

# Clean up env that confuses NDK configure tests
unset LD_LIBRARY_PATH || true
unset LIBRARY_PATH || true

export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
export ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"
export ANDROIDSDK="$ANDROID_HOME"
export ANDROID_SDK_ROOT="$ANDROID_HOME"
export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"
export PATH="$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"

ls -lh libs/arm64-v8a || true
ls -ld models || echo "⚠️ models symlink missing"

# Run, capture output, but don't stop the cell on failure.
python -m pythonforandroid.toolchain create \
  --dist_name=voiceapp \
  --bootstrap=sdl2 \
  --requirements=python3==3.10.13,kivy==2.3.0,pyjnius,vosk==0.3.45,jdatetime \
  --arch=arm64-v8a \
  --copy-libs \
  --color=always \
  --storage-dir=/content/.p4a \
  --ndk-api=21 \
  --ignore-setup-py \
  --debug 2>&1 | tee /content/p4a-create.log

# Grab the actual exit code of p4a (first element of PIPESTATUS)
RC=${PIPESTATUS[0]}
echo "=== pythonforandroid exit code: $RC ==="

echo "=== tail of the log (last 160 lines) ==="
tail -n 160 /content/p4a-create.log || true

echo "=== quick grep for likely causes ==="
grep -E "HTTP Error|C compiler cannot create executables|SdkManagerCli|not found|ERROR|failed" -n /content/p4a-create.log | tail -n 60 || true

# Always exit 0 so Colab shows the output
exit 0


total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so
lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42
/usr/bin/python3: Error while finding module specification for 'pythonforandroid.toolchain' (ModuleNotFoundError: No module named 'pythonforandroid')
=== pythonforandroid exit code: 1 ===
=== tail of the log (last 160 lines) ===
/usr/bin/python3: Error while finding module specification for 'pythonforandroid.toolchain' (ModuleNotFoundError: No module named 'pythonforandroid')
=== quick grep for likely causes ===


+ cd /content/drive/MyDrive/voiceapp/project
+ unset LD_LIBRARY_PATH
+ unset LIBRARY_PATH
+ export GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ export PATH=/content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin:/root/.local/bin:/opt/bin:/usr/local/nvi

In [ ]:
# Colab Python cell
import os
%cd /content/drive/MyDrive/voiceapp/project

# Core env
os.environ['GRADLE_USER_HOME'] = '/content/drive/MyDrive/voiceapp/cache/.gradle'
os.environ['ANDROID_HOME']     = '/content/drive/MyDrive/voiceapp/cache/android-sdk'
os.environ['ANDROIDSDK']       = os.environ['ANDROID_HOME']
os.environ['ANDROIDNDK']       = '/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b'
os.environ['JAVA_HOME']        = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH']            += f":{os.environ['ANDROID_HOME']}/tools/bin:/root/.local/bin"

# Scrub dangerous host lib paths that break NDK test links
for k in ("LD_LIBRARY_PATH","LIBRARY_PATH"):
    if k in os.environ:
        print("unset", k); del os.environ[k]

# Sanity: show models and native libs
!ls -lh libs/arm64-v8a || true
!ls -ld models || echo "⚠️ models symlink missing"

# Build (storage in /content to avoid Drive FUSE weirdness)
!python -m buildozer -v android debug --arch=arm64-v8a --storage-dir=/content/.buildozer


/content/drive/MyDrive/voiceapp/project
total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so
lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42
# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? y
# WARNING: Config token app android.sdk is deprecated and ignored, but you set value 33
# Ensure build layout
# Create directory /content/drive/MyDrive/voiceapp/project/.buildozer
# Create directory /content/drive/MyDrive/voiceapp/project/.buildozer/applibs
# Create directory /content/drive/MyDrive/voiceapp/project/.buildozer/android/platform
# Create directory /content/drive/MyDrive/voiceapp/project/.buildozer/android/app
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> foun

In [ ]:
%%bash
set -euxo pipefail
LOG=$(find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log' | head -n1 || true)
if [ -n "$LOG" ]; then
  echo "----- tail of $LOG -----"
  tail -n 200 "$LOG"
else
  echo "No libffi config.log found; build likely failed earlier than libffi."
fi


No libffi config.log found; build likely failed earlier than libffi.


++ head -n1
++ find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log'
find: ‘/content/.buildozer’: No such file or directory
++ true
+ LOG=
+ '[' -n '' ']'
+ echo 'No libffi config.log found; build likely failed earlier than libffi.'


In [ ]:
%%bash
set -euxo pipefail
LOG=$(find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log' | head -n1 || true)
if [ -n "$LOG" ]; then
  echo "----- tail of $LOG -----"
  tail -n 120 "$LOG"
else
  echo "No libffi config.log found; build likely failed earlier."
fi


No libffi config.log found; build likely failed earlier.


++ find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log'
++ head -n1
find: ‘/content/.buildozer’: No such file or directory
++ true
+ LOG=
+ '[' -n '' ']'
+ echo 'No libffi config.log found; build likely failed earlier.'


In [ ]:
%%bash
set -euxo pipefail
# list any libffi config.log produced during this run
find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log' -print || true

# If one exists, print the error section
LOG=$(find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log' | head -n1 || true)
if [ -n "$LOG" ]; then
  echo "----- $LOG (tail around error) -----"
  sed -n '1,120p' "$LOG" | sed -n '/^configure: error/,$p"'
else
  echo "No libffi config.log yet (build may have failed earlier)."
fi


No libffi config.log yet (build may have failed earlier).


+ find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log' -print
find: ‘/content/.buildozer’: No such file or directory
+ true
++ find /content/.buildozer -path '*/other_builds/libffi/*/libffi/config.log'
find: ‘/content/.buildozer’: No such file or directory
++ head -n1
++ true
+ LOG=
+ '[' -n '' ']'
+ echo 'No libffi config.log yet (build may have failed earlier).'


Fixing ModuleNotFoundError: pythonforandroid

In [ ]:
# Colab Python cell — install the *available* versions
%pip install -q --upgrade pip setuptools wheel
%pip install -q "cython<3" "buildozer>=1.5.0" "python-for-android==2024.1.21"

import sys, pythonforandroid as p4a, buildozer
print("Python:", sys.executable)
print("p4a:", p4a.__version__)
print("Buildozer:", buildozer.__version__)


Python: /usr/bin/python3
p4a: 2024.01.21
Buildozer: 1.5.0


In [ ]:
%%bash
set -euxo pipefail
SDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"

# Use the *real* cmdline-tools if you prefer (works fine):
# "${SDK}/cmdline-tools/latest/bin/sdkmanager" --sdk_root="${SDK}" --version

# If you want to keep using the legacy tools/ path, make sure env is set:
export ANDROID_SDK_ROOT="${SDK}"
export ANDROID_HOME="${SDK}"

# Optional: ensure the legacy tools/ tree has a full copy of cmdline-tools/latest
mkdir -p "${SDK}/tools"
if [ ! -x "${SDK}/tools/bin/sdkmanager" ]; then
  cp -a "${SDK}/cmdline-tools/latest/." "${SDK}/tools/"
fi

# Now sdkmanager knows the SDK root
"${SDK}/tools/bin/sdkmanager" --sdk_root="${SDK}" --version


12.0



+ SDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ mkdir -p /content/drive/MyDrive/voiceapp/cache/android-sdk/tools
+ '[' '!' -x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager ']'
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --sdk_root=/content/drive/MyDrive/voiceapp/cache/android-sdk --version


In [ ]:
%%bash
set -euxo pipefail
cd /content/drive/MyDrive/voiceapp/project

# Use the p4a you installed via pip, not a git checkout
# Comment out lines if present; ignore errors if not found
sed -i 's/^p4a\.branch *=.*/# p4a.branch (disabled to use pip python-for-android)/' buildozer.spec || true
sed -i 's/^p4a\.local_recipes *=.*/# p4a.local_recipes (disabled)/' buildozer.spec || true

# Keep requirements pinned to a real tarball version to avoid 404s
sed -i 's/^requirements *=.*/requirements = python3==3.10.13, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime/' buildozer.spec

# Quick sanity: model + native libs
ls -ld models || echo "⚠️ models symlink missing"
ls -lh libs/arm64-v8a || true


lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42
total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so


+ cd /content/drive/MyDrive/voiceapp/project
+ sed -i 's/^p4a\.branch *=.*/# p4a.branch (disabled to use pip python-for-android)/' buildozer.spec
+ sed -i 's/^p4a\.local_recipes *=.*/# p4a.local_recipes (disabled)/' buildozer.spec
+ sed -i 's/^requirements *=.*/requirements = python3==3.10.13, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime/' buildozer.spec
+ ls -ld models
+ ls -lh libs/arm64-v8a


In [ ]:
%%bash
# don't -e; we want the log even if the build fails
set -uox pipefail
cd /content/drive/MyDrive/voiceapp/project

# Clean env that confuses NDK link tests
unset LD_LIBRARY_PATH || true
unset LIBRARY_PATH || true

export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
export ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"
export ANDROID_SDK_ROOT="$ANDROID_HOME"
export ANDROIDSDK="$ANDROID_HOME"
export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"
export PATH="$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"

echo "== sdkmanager =="
sdkmanager --sdk_root="$ANDROID_HOME" --version || true

echo "== sanity =="
ls -ld models || echo "⚠ models symlink missing"
ls -lh libs/arm64-v8a || true

# show which p4a Buildozer's venv will use (if it already exists)
VENV=/root/.buildozer/venv
if [ -x "$VENV/bin/python" ]; then
  "$VENV/bin/python" - <<'PY'
import sys
try:
    import pythonforandroid as p4a
    print("p4a in buildozer venv:", p4a.__version__)
except Exception as e:
    print("p4a not yet installed in buildozer venv:", e)
PY
fi

# Run buildozer, capture full log
LOG=/content/buildozer.log
python -m buildozer -v android debug --arch=arm64-v8a --storage-dir=/content/.buildozer 2>&1 | tee "$LOG"
RC=${PIPESTATUS[0]}
echo "=== buildozer exit code: $RC ==="

echo "=== last 120 lines ==="
tail -n 120 "$LOG" || true

echo "=== quick grep for common culprits ==="
grep -En "HTTP Error|SdkManagerCli|cannot create executables|not found|ERROR|failed" "$LOG" | tail -n 80 || true

# print p4a version inside buildozer venv after it bootstraps
if [ -x "$VENV/bin/python" ]; then
  "$VENV/bin/python" - <<'PY'
try:
    import pythonforandroid as p4a
    print("p4a in buildozer venv (post):", p4a.__version__)
except Exception as e:
    print("p4a still missing in buildozer venv:", e)
PY
fi

# keep cell "successful" so we can see the log even if RC != 0
exit 0


== sdkmanager ==
12.0

== sanity ==
lrw------- 1 root root 37 Nov  1 09:52 models -> /content/models-fa/vosk-model-fa-0.42
total 14M
-rw------- 1 root root 5.5M Nov  1 08:18 librnnoise.so
-rw------- 1 root root 8.5M Feb  1  1980 libvosk.so
# Check configuration tokens
Buildozer is running as root!
This is not recommended, and may lead to problems later.
Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to continue [y/n]? Are you sure you want to cont

+ cd /content/drive/MyDrive/voiceapp/project
+ unset LD_LIBRARY_PATH
+ unset LIBRARY_PATH
+ export GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROIDSDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ export PATH=/content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin:/content/drive/MyDrive/vo

CalledProcessError: Command 'b'# don\'t -e; we want the log even if the build fails\nset -uox pipefail\ncd /content/drive/MyDrive/voiceapp/project\n\n# Clean env that confuses NDK link tests\nunset LD_LIBRARY_PATH || true\nunset LIBRARY_PATH || true\n\nexport GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"\nexport ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"\nexport ANDROID_SDK_ROOT="$ANDROID_HOME"\nexport ANDROIDSDK="$ANDROID_HOME"\nexport ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"\nexport JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"\nexport PATH="$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"\n\necho "== sdkmanager =="\nsdkmanager --sdk_root="$ANDROID_HOME" --version || true\n\necho "== sanity =="\nls -ld models || echo "\xe2\x9a\xa0 models symlink missing"\nls -lh libs/arm64-v8a || true\n\n# show which p4a Buildozer\'s venv will use (if it already exists)\nVENV=/root/.buildozer/ve

In [ ]:
%%bash
set -eux
SDK="/content/drive/MyDrive/voiceapp/cache/android-sdk"

# clean any bad copies from earlier
rm -f "${SDK}/tools/bin/sdkmanager" "${SDK}/tools/bin/avdmanager" || true
mkdir -p "${SDK}/tools/bin"

# wrapper for sdkmanager → calls the real one in cmdline-tools
cat > "${SDK}/tools/bin/sdkmanager" <<'SH'
#!/usr/bin/env bash
REAL="$(cd "$(dirname "$0")/../../cmdline-tools/latest/bin" && pwd)/sdkmanager"
exec "$REAL" "$@"
SH
chmod +x "${SDK}/tools/bin/sdkmanager"

# wrapper for avdmanager (some scripts probe for it)
cat > "${SDK}/tools/bin/avdmanager" <<'SH'
#!/usr/bin/env bash
REAL="$(cd "$(dirname "$0")/../../cmdline-tools/latest/bin" && pwd)/avdmanager"
exec "$REAL" "$@"
SH
chmod +x "${SDK}/tools/bin/avdmanager"

# prove the legacy path now works AND sees the right SDK root
"${SDK}/tools/bin/sdkmanager" --sdk_root="${SDK}" --version

# ensure required packages exist (idempotent)
yes | "${SDK}/tools/bin/sdkmanager" --sdk_root="${SDK}" --licenses || true
"${SDK}/tools/bin/sdkmanager" --sdk_root="${SDK}" \
  "platform-tools" "platforms;android-33" "build-tools;33.0.2"


12.0

[=======================================] 100% Computing updates...             
All SDK package licenses accepted.

[=======================================] 100% Computing updates...             



+ SDK=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ rm -f /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/avdmanager
+ mkdir -p /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin
+ cat
+ chmod +x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager
+ cat
+ chmod +x /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/avdmanager
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --sdk_root=/content/drive/MyDrive/voiceapp/cache/android-sdk --version
+ yes
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --sdk_root=/content/drive/MyDrive/voiceapp/cache/android-sdk --licenses
+ /content/drive/MyDrive/voiceapp/cache/android-sdk/tools/bin/sdkmanager --sdk_root=/content/drive/MyDrive/voiceapp/cache/android-sdk platform-tools 'platforms;android-33' 'build-tools;33.0.2'


In [ ]:
%%bash
set -eux
cd /content/drive/MyDrive/voiceapp/project

# keep the NDK link tests clean
unset LD_LIBRARY_PATH || true
unset LIBRARY_PATH || true

# allow root, both via env and spec (belt + suspenders)
grep -q '^\[buildozer\]' buildozer.spec || printf '\n[buildozer]\n' >> buildozer.spec
sed -i '/^allow_root *=/d' buildozer.spec
printf 'allow_root = 1\n' >> buildozer.spec
export BUILDOZER_ALLOW_ROOT=1
export BUILDOZER_WARN_ON_ROOT=0

# env
export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
export ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"
export ANDROID_SDK_ROOT="$ANDROID_HOME"
export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"
export PATH="$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"

python -m buildozer -v android debug --arch=arm64-v8a --storage-dir=/content/.buildozer \
  2>&1 | tee /content/buildozer.log || true

tail -n 150 /content/buildozer.log || true


# Check configuration tokens
# Ensure build layout
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> found at /usr/local/bin/cython
# Search for Java compiler (javac)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/javac
# Search for Java keytool (keytool)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool
# Install platform
# Run ['git', 'config', '--get', 'remote.origin.url']
# Cwd /content/drive/MyDrive/voiceapp/project/.buildozer/android/platform/python-for-android
https://github.com/kivy/python-for-android.git
# Run ['git', 'branch', '-vv']
# Cwd /content/drive/MyDrive/voiceapp/project/.buildozer/android/platform/python-for-android
* master 957a3e5f [origin/master] Merge pull request #2959 from kivy/release-2024.01.21
# Run ['/usr/bin/python3', '-m', 'pip', 'install', '-q', '--user', 'appdirs', 'colorama>=0.3.3', 'jinja2', 'sh>=1.10, <2.0; sys_platfo

+ cd /content/drive/MyDrive/voiceapp/project
+ unset LD_LIBRARY_PATH
+ unset LIBRARY_PATH
+ grep -q '^\[buildozer\]' buildozer.spec
+ sed -i '/^allow_root *=/d' buildozer.spec
+ printf 'allow_root = 1\n'
+ export BUILDOZER_ALLOW_ROOT=1
+ BUILDOZER_ALLOW_ROOT=1
+ export BUILDOZER_WARN_ON_ROOT=0
+ BUILDOZER_WARN_ON_ROOT=0
+ export GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ export PATH=/

In [ ]:
%%bash
set -eux
cd /content/drive/MyDrive/voiceapp/project

# 1) move Buildozer's build_dir off Drive
grep -q '^\[buildozer\]' buildozer.spec || printf '\n[buildozer]\n' >> buildozer.spec
sed -i '/^build_dir *=/d' buildozer.spec
printf 'build_dir = /content/.buildozer\n' >> buildozer.spec

# 2) nuke any old Drive build dirs so nothing reuses them
rm -rf .buildozer || true
rm -rf /content/.buildozer || true


+ cd /content/drive/MyDrive/voiceapp/project
+ grep -q '^\[buildozer\]' buildozer.spec
+ sed -i '/^build_dir *=/d' buildozer.spec
+ printf 'build_dir = /content/.buildozer\n'
+ rm -rf .buildozer
+ rm -rf /content/.buildozer


In [ ]:
%%bash
set -eux
cd /content/drive/MyDrive/voiceapp/project

# keep toolchain probes clean
unset LD_LIBRARY_PATH || true
unset LIBRARY_PATH || true

# no interactive root prompt
export BUILDOZER_ALLOW_ROOT=1
export BUILDOZER_WARN_ON_ROOT=0

# Android bits
export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
export ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"
export ANDROID_SDK_ROOT="$ANDROID_HOME"
export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"
export PATH="$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"

# disable ccache for this pass
export CCACHE_DISABLE=1

# run the build, keep the log
python -m buildozer -v android debug --arch=arm64-v8a --storage-dir=/content/.buildozer \
  2>&1 | tee /content/buildozer.log || true

# quick sanity: libffi should now be under /content/.buildozer
grep -n "other_builds/libffi" /content/buildozer.log | tail -n 5 || true

# show the tail
tail -n 120 /content/buildozer.log || true

# Check configuration tokens
# Ensure build layout
# Create directory /content/.buildozer
# Create directory /content/.buildozer/applibs
# Create directory /content/.buildozer/android/platform
# Create directory /content/.buildozer/android/app
# Check configuration tokens
# Preparing build
# Check requirements for android
# Search for Git (git)
#  -> found at /usr/bin/git
# Search for Cython (cython)
#  -> found at /usr/local/bin/cython
# Search for Java compiler (javac)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/javac
# Search for Java keytool (keytool)
#  -> found at /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool
# Install platform
# Run ['git', 'clone', '-b', 'master', '--single-branch', 'https://github.com/kivy/python-for-android.git', 'python-for-android']
# Cwd /content/.buildozer/android/platform
Cloning into 'python-for-android'...
# Run ['/usr/bin/python3', '-m', 'pip', 'install', '-q', '--user', 'appdirs', 'colorama>=0.3.3', 'jinja2', 'sh>=1.10, <2.0; sys_platform!

+ cd /content/drive/MyDrive/voiceapp/project
+ unset LD_LIBRARY_PATH
+ unset LIBRARY_PATH
+ export BUILDOZER_ALLOW_ROOT=1
+ BUILDOZER_ALLOW_ROOT=1
+ export BUILDOZER_WARN_ON_ROOT=0
+ BUILDOZER_WARN_ON_ROOT=0
+ export GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ GRADLE_USER_HOME=/content/drive/MyDrive/voiceapp/cache/.gradle
+ export ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_HOME=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ ANDROID_SDK_ROOT=/content/drive/MyDrive/voiceapp/cache/android-sdk
+ export ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ ANDROIDNDK=/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b
+ export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
+ export PATH=/content/drive/MyDrive/voiceapp/cache/android-sdk/cmdline-tools/latest/bin:/content/drive/MyDrive/voiceapp/cache/an

In [ ]:
%%bash
set -eux
cd /content/drive/MyDrive/voiceapp/project

unset LD_LIBRARY_PATH || true
unset LIBRARY_PATH || true

export BUILDOZER_ALLOW_ROOT=1
export BUILDOZER_WARN_ON_ROOT=0

export GRADLE_USER_HOME="/content/drive/MyDrive/voiceapp/cache/.gradle"
export ANDROID_HOME="/content/drive/MyDrive/voiceapp/cache/android-sdk"
export ANDROID_SDK_ROOT="$ANDROID_HOME"
export ANDROIDNDK="/content/drive/MyDrive/voiceapp/cache/android-ndk-r25b"
export JAVA_HOME="/usr/lib/jvm/java-17-openjdk-amd64"
export PATH="$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/tools/bin:/root/.local/bin:$PATH"

export CCACHE_DISABLE=1   # <— important for this pass

# Make sure CPython version is exact to avoid 404s
sed -i 's/^requirements *=.*/requirements = python3==3.10.13, kivy==2.3.0, pyjnius, vosk==0.3.45, jdatetime/' buildozer.spec

python -m buildozer -v android debug --arch=arm64-v8a 2>&1 | tee /content/buildozer.log || true
tail -n 150 /content/buildozer.log || true


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving api_helpers.py to api_helpers.py
Saving audio_engine.py to audio_engine.py
Saving buildozer.spec to buildozer.spec
Saving camera_popup.py to camera_popup.py
Saving main.kv to main.kv
Saving main.py to main.py
Saving pyaudio_stub.py to pyaudio_stub.py
Saving used_bars.json to used_bars.json
Saving utils_persian.py to utils_persian.py


In [ ]:
!ls

api_helpers.py	 include	main.py		 third_party
audio_engine.py  jni		models		 used_bars.json
bin		 librnnoise.so	pyaudio_stub.py  utils_persian.py
buildozer.spec	 libs		recipes		 vosk-android-0.3.45.aar
camera_popup.py  main.kv	src
